# Lab-10-1 Convolution

In [1]:
import torch
import torch.nn as nn
# in_channels, out_channels, kernel_size(filter)
conv = nn.Conv2d(1, 1, 3, stride = 1, padding=1)
conv

Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [2]:
inputs = torch.Tensor(1, 1, 64, 32)
inputs.shape

torch.Size([1, 1, 64, 32])

In [3]:
out = conv(inputs)
out.shape

torch.Size([1, 1, 64, 32])

In [4]:
input = torch.Tensor(1, 1, 28, 28)
conv1 = nn.Conv2d(1, 5, 5)


In [5]:
pool = nn.MaxPool2d(2)
out = conv1(input)
out2 = pool(out)

In [6]:
out.size()

torch.Size([1, 5, 24, 24])

In [7]:
out2.size()

torch.Size([1, 5, 12, 12])

# Lab-10-2 mnist cnn

In [8]:
inputs = torch.Tensor(1, 1, 28, 28)

In [9]:
inputs.shape

torch.Size([1, 1, 28, 28])

In [10]:
conv1 = nn.Conv2d(1, 32, 3, padding = 1)
pooling = nn.MaxPool2d(2)

In [11]:
conv2 = nn.Conv2d(32, 64, 3, padding = 1)

In [12]:
conv1, conv2

(Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))

In [13]:
out = conv1(inputs)
out = pooling(out)
out.shape

torch.Size([1, 32, 14, 14])

In [14]:
out = conv2(out)
out = pooling(out)
out.shape

torch.Size([1, 64, 7, 7])

In [15]:
out = out.view(out.size(0), -1)

In [16]:
out.shape

torch.Size([1, 3136])

In [17]:
fc = nn.Linear(3136, 10)
out = fc(out)
out.shape

torch.Size([1, 10])

In [18]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [20]:
print(device)

cuda


In [21]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [22]:
# MNIST dataset

mnist_train = dsets.MNIST(root = "MNIST_data/", train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = "MNIST_data/", train = False,
                          transform = transforms.ToTensor(),
                          download = True)



Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [23]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)



In [56]:

class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(3 * 3 * 128, 625)
        self.relu = nn.ReLU()
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)  
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)

        
        return out

In [58]:
model = CNN().to(device)

value = torch.Tensor(1, 1, 28, 28).to(device)
print(model(value).shape)

torch.Size([1, 10])


In [52]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2048, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

In [59]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [60]:
# training

total_batch = len(data_loader)
print("Learning Start")

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)

        cost = criterion(hypothesis, Y)
        cost.backward()

        optimizer.step()

        avg_cost += cost / total_batch
    
    print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))

print("Learning Finished")

Learning Start
[Epoch: 1] cost = 0.16978269815444946
[Epoch: 2] cost = 0.04362749308347702
[Epoch: 3] cost = 0.03079836815595627
[Epoch: 4] cost = 0.022284528240561485
[Epoch: 5] cost = 0.017833001911640167
[Epoch: 6] cost = 0.016658684238791466
[Epoch: 7] cost = 0.012010172940790653
[Epoch: 8] cost = 0.01090269349515438
[Epoch: 9] cost = 0.00956809427589178
[Epoch: 10] cost = 0.007948646321892738
[Epoch: 11] cost = 0.007099435664713383
[Epoch: 12] cost = 0.007394263055175543
[Epoch: 13] cost = 0.007845419459044933
[Epoch: 14] cost = 0.004055491648614407
[Epoch: 15] cost = 0.006405810359865427
Learning Finished


In [61]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy: ", accuracy.item())

Accuracy:  0.9888999462127686


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


# Lab-10-3 visdom

Cannot use in colab :(

- https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-10_3_1_visdom-example.ipynb

- https://www.youtube.com/watch?v=bAPzUjfPAlQ&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=21